In [5]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [6]:
# load the model, scaler and one hot encoder
import pickle
from keras import models
from keras.models import load_model

model = load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


In [7]:
# example input data

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [9]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
# one hot encode "Geography"
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns= label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [10]:
# encode categorical variables
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [11]:
# concatention one hot encoded data 
input_data = pd.concat([input_data.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
# scaling the input data
scaled_input = scaler.transform(input_data)
scaled_input

array([[-0.51795799,  0.90682052,  0.10698342, -0.69673114, -0.26352876,
         0.79286681,  0.64298333,  0.97067965, -0.86629511,  1.002002  ,
        -0.57946723, -0.57677292]])

In [13]:
# prediction 
prediction = model.predict(scaled_input)
prediction

1/1 [==============================] - 0s 187ms/step


array([[0.04346325]], dtype=float32)

In [14]:
prediction_prob = prediction[0][0]
prediction_prob

0.043463245

In [15]:
if prediction_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
